In [1]:
import surprise
from surprise import Dataset, Reader
from surprise import model_selection, print_perf
from surprise import SVD

## SVD

FunkSVD和BiasSVD的思想实现
SVD的参数参考：http://surprise.readthedocs.io/en/stable/matrix_factorization.html#surprise.prediction_algorithms.matrix_factorization.SVD
主要参数：
1. n_factors：给定隐因子的数目，默认为100
2. n_epochs：梯度下降的迭代次数，默认为20
3. biased：给定模型中是否使用偏置项，默认为True，即BiasSVD分解模型；设置为False，表示用FunkSVD
4. lr_all：给定学习率，全部参数使用相同的学习率，默认为0.005
5. reg_all：给定正则化系数，全部参数使用相同的正则化系数，默认为0.02

## SVDpp
SVD++算法的思想实现

## NMF
非负矩阵分解的思想实现<br/>
备注:也支持bias偏置项的转换

In [2]:
# 1. 加载数据
data = Dataset.load_builtin('ml-100k')

# 2. 将数据转换为Dataset训练集
dataset = data.build_full_trainset()


# 模型对象构建
algo = SVD(n_factors=10, n_epochs=10, reg_all=0.2, lr_all=0.05)

# TODO: 在模型训练之前，一定需要将数据进行build成为训练集, data.build_full_trainset
# 模型训练
algo.fit(dataset)

# <br/>

## 算法模型调参

也就是给定一个合适的超参；同样使用类似sklearn中的网格交叉验证（GridSearchCV）来选择最优的参数

### 如果安装的是numpy+mkl的库的话，必须将numpy的core文件中所有以mkl_开头的动态链接库(dll文件)放到根目录(即Python.exe所在的文件夹)

In [4]:
from surprise import GridSearch
from surprise import SVD

# 定义可选的参数列表
param_grid = {
    'n_epochs': [10, 20], # 迭代次数
    'lr_all': [0.1, 0.5], # 学习率
    'reg_all': [0.1, 0.5] # 正则化系数
}

# 定义网格搜索的对象
# NOTE: 第一个参数不是算法对象，而是算法所对应的Class
grid_search = model_selection.GridSearchCV(algo_class=SVD, param_grid=param_grid, measures=['rmse', 'mae', 'fcp'], cv=5)

# 加载数据&数据进行K-Flod
# 进行模型训练
grid_search.fit(data)

# 输出调优的参数列表
## 输出最优的RMSE的值
print(grid_search.best_score['rmse'])
print(grid_search.best_params['rmse'])

## 输出最优的MAE的值
print(grid_search.best_score['mae'])
print(grid_search.best_params['mae'])

## 输出最优的FCP的值
print(grid_search.best_score['fcp'])
print(grid_search.best_params['fcp'])

0.9486001452404732
{'n_epochs': 20, 'lr_all': 0.1, 'reg_all': 0.1}
0.7491353886691188
{'n_epochs': 10, 'lr_all': 0.1, 'reg_all': 0.1}
0.6999145004694014
{'n_epochs': 20, 'lr_all': 0.1, 'reg_all': 0.1}


# <br/>

## 模型的持久化
1. 可以将模型的预测结果持久化,直接调用algo.predict获取用户对物品的评分，并将这些评分输出保存；
2. 将模型保存成为文件的形式，然后在需要的程序中再加载进来

In [5]:
import surprise

## 模型保存
surprise.dump.dump('./model/svd.recommender.model', algo=algo)

# 模型加载使用
algo2 = surprise.dump.load('./model/svd.recommender.model')[1]
print("用户1对于物品10的评分为:%.3f" % algo2.predict('1', '10').est)

用户1对于物品10的评分为:4.060
